# 🎯 Objective
This notebook processes raw Reddit JSON data, cleans it, and saves it into a structured SQLite database. 
We will create two linked tables: 'posts' and 'comments'.




# 📚 Libraries




In [20]:
import os  # File operations
import json  # Handling JSON data
import sqlite3  # Database storage
import pandas as pd  # Data processing & transformation
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# 📊 Data Loading

In [21]:
# Define correct file paths
BASE_DIR = "/files/ds105a-2024-alternative-summative-ajchan03"  
DATA_DIR = os.path.join(BASE_DIR, "data", "raw")  

# Define file paths
POSTS_FILE = os.path.join(DATA_DIR, "reddit_filtered_posts.json")
COMMENTS_FILE = os.path.join(DATA_DIR, "reddit_filtered_comments.json")

# Check if JSON files exist before attempting to load them
if not os.path.exists(POSTS_FILE):
    raise FileNotFoundError(f"🚨 Error: `{POSTS_FILE}` not found. Please run the scraper first.")

if not os.path.exists(COMMENTS_FILE):
    raise FileNotFoundError(f"🚨 Error: `{COMMENTS_FILE}` not found. Please run the scraper first.")

# Load JSON data into DataFrames
with open(POSTS_FILE, "r", encoding="utf-8") as f:
    posts_data = json.load(f)
df_posts = pd.DataFrame(posts_data)

with open(COMMENTS_FILE, "r", encoding="utf-8") as f:
    comments_data = json.load(f)
df_comments = pd.DataFrame(comments_data)

# Data successfully loaded
print("Data Loaded Successfully")


Data Loaded Successfully


# 🧹 Data Cleaning & Transformation

After loading the raw data, we need to **clean and transform it** to ensure its quality.

### ✅ **Key Cleaning Tasks**
1️⃣ **Handle missing values** → Fill or remove missing fields  
2️⃣ **Convert data types** → Ensure `created_utc` fields are in `datetime` format  
3️⃣ **Remove duplicates** → Ensure each post and comment is unique  
4️⃣ **Ensure data consistency** → Ensure all comments link to valid posts  
5️⃣ **Perform sentiment analysis** → Use **NLTK's VADER** to assign sentiment scores to each comment  

### 📌 **New Features Added**
- **`comment_sentiment`** → A numerical score representing the sentiment of a comment.  
  - **Positive sentiment** → Score **> 0.05**
  - **Neutral sentiment** → Score **between -0.05 and 0.05**
  - **Negative sentiment** → Score **< -0.05**  

By the end of this step, our data will be **fully cleaned and enhanced with sentiment scores** for deeper analysis in the next steps.


In [22]:
# 🚀 Step 2: Data Cleaning & Sentiment Analysis



# Download the VADER sentiment analysis tool
nltk.download("vader_lexicon")

# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# 🧹 Handle Missing Values
df_posts.fillna("", inplace=True)
df_comments.fillna("", inplace=True)

# ✅ Convert Date Fields to Datetime
df_posts["created_utc"] = pd.to_datetime(df_posts["created_utc"])
df_comments["created_utc"] = pd.to_datetime(df_comments["created_utc"])

# ✅ Remove Duplicate Posts
initial_post_count = len(df_posts)
df_posts.drop_duplicates(subset=["id"], keep="first", inplace=True)
print(f"\n✅ Removed {initial_post_count - len(df_posts)} duplicate posts")

# ✅ Remove Duplicate Comments
initial_comment_count = len(df_comments)
df_comments.drop_duplicates(subset=["comment_id"], keep="first", inplace=True)
print(f"\n✅ Removed {initial_comment_count - len(df_comments)} duplicate comments")

# ✅ Ensure Foreign Key Consistency
df_comments = df_comments[df_comments["post_id"].isin(df_posts["id"])]
print(f"\n✅ Foreign Key Consistency Check: {len(df_comments)} comments remain after filtering")

# ✅ Add Sentiment Analysis for Comments
df_comments["comment_sentiment"] = df_comments["body"].apply(lambda text: sia.polarity_scores(text)["compound"])

print("\n✅ Added Sentiment Score: `comment_sentiment` for comments")

# ✅ Final Data Validation
print("\n🔎 Final Missing Value Check (Should be 0s)")
print(df_posts.isnull().sum())
print(df_comments.isnull().sum())

print("\n📊 Final Data Shape")
print(f"Posts: {df_posts.shape}")
print(f"Comments: {df_comments.shape}")

# ✅ Display Cleaned Data Samples
print("\n📊 Cleaned Comments Data Sample with Sentiment Score:")
print(df_comments[["body", "comment_sentiment"]].head())

print("\n✅ Step 2: Data Cleaning & Sentiment Analysis Completed!")



✅ Removed 51 duplicate posts

✅ Removed 3769 duplicate comments

✅ Foreign Key Consistency Check: 3403 comments remain after filtering


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/datahub/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!



✅ Added Sentiment Score: `comment_sentiment` for comments

🔎 Final Missing Value Check (Should be 0s)
id              0
subreddit       0
title           0
score           0
num_comments    0
created_utc     0
text            0
url             0
dtype: int64
post_id              0
comment_id           0
body                 0
score                0
created_utc          0
comment_sentiment    0
dtype: int64

📊 Final Data Shape
Posts: (49, 8)
Comments: (3403, 6)

📊 Cleaned Comments Data Sample with Sentiment Score:
                                                body  comment_sentiment
0     Does this mean he can’t even vote for himself?             0.0000
1  I wonder if Trump is regretting the " Lock her...             0.5101
2                        He won 100% of the vote lol             0.7579
3  He fucked a porn star. Then, he got fucked by ...            -0.8689
4  Remember when he said Hillary should drop out ...            -0.2732

✅ Step 2: Data Cleaning & Sentiment Analysis Co

# 💾 Database Design

Now that our **data is cleaned** and **sentiment scores are added**, we will:
- **Define the SQLite database structure**
- **Create tables (`posts` & `comments`) with foreign key relationships**
- **Store the cleaned data into the database**

### **Database Structure**
We will store the data in **`data/reddit_data.db`**.

| **Table**   | **Columns** | **Primary Key** | **Foreign Key** |
|------------|------------|----------------|----------------|
| **posts**  | `id, subreddit, title, score, num_comments, created_utc, text, url` | `id` | - |
| **comments** | `comment_id, post_id, body, score, created_utc, comment_sentiment` | `comment_id` | `post_id` (FK → posts.id) |

---



# 📥 Database Creation

In [23]:
# 📥 Step 4: Database Creation

# Define database path
DB_PATH = os.path.join(BASE_DIR, "data", "reddit_data.db")

# Connect to SQLite and create tables
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Create 'posts' table
cursor.execute("""
CREATE TABLE IF NOT EXISTS posts (
    id TEXT PRIMARY KEY,
    subreddit TEXT,
    title TEXT,
    score INTEGER,
    num_comments INTEGER,
    created_utc DATETIME,
    text TEXT,
    url TEXT
);
""")

# Create 'comments' table with sentiment analysis
cursor.execute("""
CREATE TABLE IF NOT EXISTS comments (
    comment_id TEXT PRIMARY KEY,
    post_id TEXT,
    body TEXT,
    score INTEGER,
    created_utc DATETIME,
    comment_sentiment REAL,
    FOREIGN KEY (post_id) REFERENCES posts (id)
);
""")

# Insert data into SQLite database
df_posts.to_sql("posts", conn, if_exists="replace", index=False)
df_comments.to_sql("comments", conn, if_exists="replace", index=False)

conn.commit()
print("\n✅ Database Creation & Data Insertion Completed!")



✅ Database Creation & Data Insertion Completed!


## Quality Check

Before moving to analysis, we ensure **database integrity** by:
- Checking record counts
- Validating foreign key relationships
- Inspecting sentiment score distribution

This ensures **clean, structured data** for analysis in NB03.


In [24]:
# ✅ Step 5: Quality Check

# Check Table Row Counts
print("\n📊 Table Row Counts:")
print("Posts:", pd.read_sql_query("SELECT COUNT(*) FROM posts;", conn).iloc[0, 0])
print("Comments:", pd.read_sql_query("SELECT COUNT(*) FROM comments;", conn).iloc[0, 0])

# Validate Foreign Keys (Ensure All Comments Link to Valid Posts)
invalid_comments = pd.read_sql_query("""
    SELECT COUNT(*) FROM comments c
    LEFT JOIN posts p ON c.post_id = p.id
    WHERE p.id IS NULL;
""", conn).iloc[0, 0]

if invalid_comments == 0:
    print("\n✅ Foreign Key Check: All comments have valid posts.")
else:
    print(f"\n⚠️ Warning: {invalid_comments} comments have no associated post!")

# Close Database Connection
conn.close()
print("\n✅ Data Quality Check Completed!")



📊 Table Row Counts:
Posts: 49
Comments: 3403

✅ Foreign Key Check: All comments have valid posts.

✅ Data Quality Check Completed!
